In [ ]:
!pip install langchain_community langchain_groq youtube_transcript_api faiss-cpu

In [ ]:
from langchain.document_loaders import YoutubeLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.vectorstores import FAISS
from langchain.chains import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
import textwrap

In [ ]:
embeddings = HuggingFaceEmbeddings()

<ipython-input-15-d0c9174021d8>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


In [ ]:
def create_db_from_youtube_video_url(video_url):
  loader = YoutubeLoader.from_youtube_url(video_url)
  transcript = loader.load()

  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap = 100)
  docs = text_splitter.split_documents(transcript)
  db = FAISS.from_documents(docs, embeddings)
  return db

In [ ]:
def get_response_from_query(db, query, k = 4):
  docs = db.similarity_search(query, k = k)
  docs_page_content = " ".join([d.page_content for d in docs])

  chat = ChatGroq(api_key = "YOUR_API_KEY", model = "llama-3.3-70b-versatile", temperature=0)

  template = """You are a helpful assistant that that can answer questions about youtube videos
        based on the video's transcript: {docs}

        Only use the factual information from the transcript to answer the question.

        If you feel like you don't have enough information to answer the question, say "I don't know"."""

  system_message_prompt = SystemMessagePromptTemplate.from_template(template)
  human_template = "Answer the following question: {question}"

  human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

  chat_prompt = ChatPromptTemplate.from_messages(
      [system_message_prompt, human_message_prompt]
  )

  chain = LLMChain(llm = chat, prompt = chat_prompt)

  response = chain.run(question = query, docs = docs_page_content)

  response = response.replace("\n", "")

  return response, docs


In [ ]:
video_url = "https://youtu.be/sJ9kSzMbMRY?si=k4dnmxrt-yMkuSAy"
db = create_db_from_youtube_video_url(video_url)
query = "what is this video about?"
response, docs = get_response_from_query(db, query)
print(textwrap.fill(response, width = 70))

<ipython-input-17-cff13e7348cc>:23: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm = chat, prompt = chat_prompt)
<ipython-input-17-cff13e7348cc>:25: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run(question = query, docs = docs_page_content)


This video is about creating an AI code review application from
scratch, including setting up the backend using Node.js and Express,
and creating the frontend using React.js. The application uses a Deep
R1 model to review and improve code in languages such as Python, Java,
and C++.


In [ ]:
print(docs)

[Document(id='e9838091-f6df-48e3-8873-56a4cb622a39', metadata={'source': 'sJ9kSzMbMRY'}, page_content="out because our code is uploaded based on the Transformers so you can also editing the code on here you can selecting all the code you can deleting them you can that I can also delete them and I I can also running here my own code so let's say def sum let's say I'm going to give it here let's say a b and I am trying to returning here let's say uh returning here A plus b and this is completely a wrong code I just give it here the review the code and TR to waiting for the few minutes or few seconds it will give me the response so well it give me the response again P tag that's me DC car1 and if I go down you can see here it will try to improving the code it will adding the coma as you as you to try to adding here some try and expert block and also to actually improve the code in the Java C and the C++ as well as it will give you time complexity asess complexity as well as it will give y

In [ ]:
#Homework to create a gradio/flask/fastapi/react-js application